#### Краткое описание:
В данном jupyter notebook представлена процедура тематического моделирования публикаций,которые были выгружены из выборки сообществ и отобраны по маркерным словам вакцинации от Covid-19.

### 1. Импорт необходимых библиотек

In [1]:
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing as mp
from top2vec import Top2Vec
nltk.download('stopwords')
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import pymorphy2
import nltk
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\polin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 2. Загрузка данных 

Описание данных:
    
    маркерные слова, по которым была отобрана публикация;
    идентифицирующие и цифровые характеристики публикации;
    размеченные данные топиков, полученные на предыдущих итерациях.
        

Данные представлены по ссылке: https://docs.google.com/spreadsheets/d/1E2XgCdbRKVWT-7CqtJYBDoAGpYZnJ8H2/edit?usp=sharing&ouid=115087076194194313473&rtpof=true&sd=true

In [2]:
df = pd.read_excel('cleared_data_finall.xlsx')

### 3. Предварительная обработка

    1. Убираем нерелевантные символы, которые могли образоваться при выгрузке данных (знак \n - перевод строки);
    2. Убираем ссылки;
    3. Убираем упоминания и хештеги;
    4. Убираем лишние пробелы, которые могли образоваться при описанных выше этапах предварительной обработки;
    5. Оставляем толькое буквенные символы латиницы и кириллицы (то есть удаляем пунктуационные знаки, эмодзи и др.)
    6. Переводим тексты в нижний регистр;
    7. Проводим лемматизацию;
    8. Удаляем стоп-слова. 

#### Формирование списка стоп-слов

Список стоп-слов состоит из базовых стоп-слов библиотеки nltk для русского и английского языка, а также дополнен стоп-словами, специфичными для данной выборки текстов и выделенными итерационным путем в процессе тестирования моделей.

In [16]:
mystopwords = ['и','в','во','что','он','на', 'я','с','со','как','а','то','все','она','так','его','но','да','ты','к','у','же','вы','за','бы','по',
               'только','ее','мне','было','вот','от','меня','еще','нет','о','из',
               'ему','теперь','когда','даже','ну','вдруг','ли','если','уже','или',
               'ни','быть','был','него','до','вас','нибудь','опять','уж','вам','ведь', 'восходинфо'
               'там','потом','себя','ничего','ей','может', 'они','тут','где','есть',
               'надо','ней','для','мы','тебя','их','чем','была','сам','чтоб','без',
               'будто','чего','раз','тоже','себе','под','будет','ж','тогда','кто',
               'этот','того','потому','этого','какой','совсем','ним','здесь','этом',
               'один','почти','мой', 'тем','чтобы', 'нее','сейчас','были','куда',
               'зачем','всех','никогда','можно','при','наконец','два','об','другой',
               'хоть','после','над','больше','тот','через','эти','нас','про','всего',
               'них','какая','много','разве','три','эту','моя','впрочем','хорошо',
               'свою','этой','перед','иногда','лучше','чуть','том','нельзя','такой',
               'им','более','всегда','конечно','всю','между', 'анон', 'анонимно', 'админ', 'пожалуйста', 
              'спасибо', 'подсказать', 'главные', 'подробный','курьер','среда', 'новостибердск', 'курьерсредабердск','лента', 'новость','интерфакс','фото',
              'событие', 'день', 'белгородская', 'область', 'последний', 'сутки','текст', 'утро',
              'сайт','риа', 'январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль',
              'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь', 'понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье',
              'территориальные', 'единица', 'вацап', 'здравствуйте', 'ксерокопия', 'пресса', 'вполне', 'прикамье',   
              'подробность', 'объявление', 'объявления', 'топ', 'news', 'друзья', 'друг', 'неделя', 'post'
               'месяц', 'обзор', 'каждый', 'читать', 'мывместе','донецк','дан','сми', 'объявление', 'внимание',
               'спасибо', 'отзывы', 'привет', 'весь', 'подсказать', 'уважаемый', 'подробность', 'главное', 'добрый', 'друг', 'подробный', 
               'напомнить', 'карточка', 'анонс', 'это', 'год', 'голос', 'мордора', 'оперативный',
               'пн', "вт", "ср", 'чт', "пт", "сб", 'вс', "ул", "фев", 'буднии', "каб", "четвергс", 'ria', "cnn", "id", 'club', "воскресении", "рбк", 'тасс', "www", "vk",  
               'иа', "instagram", "igm", 'bev', "атор", "ема", 'россиятекст', "россиянтекст", "kraj",
               'i','me','my','myself','we','our','ours','ourselves','you', "you're","you've", "you'll""you'd",'your',
               'yours','yourself','yourselves','he','him', 'his','himself','she',"she's",'her','hers','herself','it',"it's",'its','itself','they','them','their','theirs',
               'themselves','what','which','who','whom','this','that', "that'll",'these','those','am', 'is','are', 'was','were','be',
               'been','being','have','has','had','having','do','does','did','doing','a','an','the', 'and','but',
               'if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through', 'during','before','after','above','below','to','from','up',
               'down','in','out','on','off','over','under','again','further','then','once','here','there','when','where',
               'why','how', 'all', 'any', 'both','each','few','more','most','other','some','such', 'no','nor','not', 'only',
               'own','same','so','than','too','very','s','t','can','will','just','don',"don't", 'should', "should've", 'now',
               'd','ll','m', 'o','re','ve','y','ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",
               'doesn', "doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'ma', 'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't", 'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn', "wouldn't"]

In [18]:
#создаем экземпляр лемматизатора
m = MorphAnalyzer()

#отставляем в сообщениях только релевантные букувенные символы с помощью регулярных выражений и переводим текст в нижний регистр
def words_only(text):
    try:
        #знак \n - он в целом обозначает - перевод строки, но в данных он представлен текстом
        text = re.sub('\\n', '', text)
        #убираем ссылки
        text = re.sub(r'https?://\S+', '', text)
        #убираем упоминания и хештеги
        text = re.sub('#[A-ZА-Яa-zа-я0-9_]{1,50}|@[A-ZА-Яa-zа-я0-9_]{1,50}', '', text)
        #убираем лишние пробелы
        text = re.sub('\s+', ' ', text)
        #оставляем только буквенные символы кириллицы и латиницы и переводим их в нижний регистр
        return re.findall("[A-ZА-Яa-zа-яё]+", text.lower())
    except:
        return []
    
#лемматизация
def lemmatize_word (token, pymorphy = m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text (text):
    return [lemmatize_word(w) for w in text]

#удаление стоп-слов
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords]

#объединение функций в одну функцию
def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    return ' '.join(remove_stopwords(lemmas))

In [19]:
#процедура предварительной обработки данных
preprocessed_text = list(tqdm(map(clean_text, df['post_text']), total=len(df)))

  0%|          | 0/17026 [00:00<?, ?it/s]

In [20]:
#создание столбца с полученными очищенными данными
df['preprocessed_text'] = preprocessed_text

In [50]:
#удаление возможных образовавшихся пропусков при предварительной обработке
df.dropna(subset=['preprocessed_text'], inplace = True)

In [4]:
#удаляем из выборки, обанруженный на прерыдущих итерациях топик про животных (не связан с темой вакцинации от Covid-19)
df = df.loc[df['topic_finall_dv17'] != 55]

In [5]:
#приведение данных в вид, пригодный для используемой модели
docs_prep = list(df.loc[:, 'preprocessed_text'].values)

### 4. Тематическое моделирование

    модель - Top2vec;
    векторные представления -doc2vec; 
    гиперпараметры модели: workers=6, min_count = 50, speed = 'deep-learn'

In [6]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    model_dv19 = Top2Vec(docs_prep, embedding_model='doc2vec', workers=6, min_count = 50, speed = 'deep-learn')

2021-12-10 10:35:40,130 - top2vec - INFO - Pre-processing documents for training
2021-12-10 10:35:53,853 - top2vec - INFO - Creating joint document/word embedding
2021-12-10 11:45:58,697 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-10 11:46:45,065 - top2vec - INFO - Finding dense areas of documents
2021-12-10 11:46:50,527 - top2vec - INFO - Finding topics


In [7]:
#получение количества тем
n = model_dv19.get_num_topics()
print(n)

178


In [10]:
#получение номеров топиков для постов
topic = model_dv19.doc_top
df['topic_finall_dv19 '] =  topic

In [9]:
#сохранение модели 
model_dv19.save("model_dv19")

In [11]:
#сохранение полученных данных
df.to_excel('cleared_data_finall.xlsx')

### 5. Валидация модели

Выделенные топики прошли экспертную оценку, после чего были выделены невалидные топики и проведено ручное укрупнение топиков.

#### Очищаем данные от невалидной информации 

In [ ]:
df = df.loc[df['topic_finall_dv19 '] != 2]
df = df.loc[df['topic_finall_dv19 '] != 27]
df = df.loc[df['topic_finall_dv19 '] != 50]
df = df.loc[df['topic_finall_dv19 '] != 66]
df = df.loc[df['topic_finall_dv19 '] != 123]
df = df.loc[df['topic_finall_dv19 '] != 128]
df = df.loc[df['topic_finall_dv19 '] != 147]
df = df.loc[df['topic_finall_dv19 '] != 149]

#### Укрупнение категорий

In [ ]:
#укрупнение категорий
dict_dv19 = {0:1, 1:2, 79:2, 81:2, 93:2, 4:3, 20:3, 29:3, 49:3, 95:3, 131:3, 132:3, 5:4, 21:4,45:4, 64:4, 73:4, 83:4, 98:4, 112:4, 121:4, 133:4,
            6:5, 15:5, 63:5, 70:5, 104:5, 148:5, 158:5, 159:5, 171:5, 177:5, 7:6, 40:6, 61:6, 8:7, 13:7, 62:7, 110:7, 113:7, 170:7,
            9:8,10:9, 11:10, 140:10, 141:11, 55:12, 120:12, 12:12, 16:13, 91:13, 144:13, 17:14,26:14, 28:14, 58:14, 69:14, 72:14, 114:14,
            135:14, 142:14, 152:14, 154:14, 19:15, 153:15, 160:15, 168:15, 172:15, 161:16, 155:17, 167:17, 176:17, 156:18, 22:19,
            23:20, 24:20, 42:20, 51:20, 92:20, 101:20, 143:20, 111:20, 115:20, 139:20, 145:20, 150:20, 174:21, 30:22, 57:22, 105:22, 31:23, 32:24,
            33:25, 18:25, 35:25, 67:25, 166:25, 34:26, 84:26, 108:26, 136:26, 137:27, 37:28, 96:28, 41:29, 43:30, 44:31, 46:32,
            47:33, 52:34, 88:34, 89:34, 117:34, 146:34, 82:35, 53:36, 71:36, 77:36, 99:36, 100:36, 106:36, 116:36, 122:36, 129:36, 163:36, 169:36,
            60:37, 68:37, 164:37, 175:37, 56:38, 65:39, 54:39, 125:39, 74:40, 78:40, 97:40, 103:41, 80:42, 85:43, 90:44, 94:45,
            109:46, 119:46, 173:47, 118:48, 130:49, 3:50, 59:50, 157:50, 14:51, 36:52, 87:53,107:54, 124:54, 151:54, 126:55, 165:55, 127:56,
            75:57, 134:57, 38:57, 86:57, 76:58, 138:59, 162:60, 102:61, 39:62, 48:62, 25:3}

In [ ]:
#создание словаря с названием топиков
dict_categorydv19 = {1: 'Мутация вируса SARS-CoV-2', 
                     2: 'Обсуждение вакцинации пользователями', 
                     3: 'Разработка вакцины',
                     4: 'Внешняя политика (международные отношения)',
                     5: 'Пункты вакцинации', 
                     6: 'Последствия вакцинации', 
                     7: 'Туризм и путешествия', 
                     8: 'Зарубежные вакцины', 
                     9: 'Вакцинация и репродуктивное здоровье',
                     10: 'Отстранение от работы',
                     11: 'Вакцинация военных', 
                     12: 'Доступность вакцин (доставка вакцин)', 
                     13: 'Эпидемиологические ограничения',
                     14: 'Антивакерский топик', 
                     15: 'Вакансии',
                     16: 'Поощрение за прививку',
                     17: 'Эмиграция', 
                     18: 'Позиция верующих', 
                     19: 'Экономика', 
                     20: 'Общая информация о вакцинации', 
                     21: 'Вакцинация от Covid-19 и гриппа', 
                     22: 'Принудительная вакцинация',
                     23: 'Призы и розыгрыши', 
                     24: 'Общественный отношение (социологические опросы)',
                     25: 'Вакцинация в других странах',
                     26: 'Вакцинация в СМИ и социальных медиа', 
                     27: 'Вакцинация животных от Covid-19', 
                     28: 'Официальная позиция Министерства здравоохранения',
                     29: 'Ревакцинация',
                     30: 'Официальная позиция Кремля',
                     31: 'Диагностика Covid-19',
                     32 :'События, связанные с публичными личностями',
                     33: 'Профилактика Covid-19',
                     34 :'Провакерский топик',
                     35: 'Борьба с ложной информацией',
                     36 :'Мнения и высказывания экспертов', 
                     37: 'Общественные места',
                     38 : 'Правонарушения',
                     39: 'Эффективность вакцины',
                     40 :'Covid-паспорта и QR-коды',
                     41: 'Алкоголь при вакцинации',
                     42 :'Политические партии',
                     43: 'Вакцинации детей и подростков',
                     44: "Вакцинация в высших учебных заведениях",
                     45: "Спорт и спортивные события",
                     46: "Вакцинация в школьных и дошкольных учреждениях",
                     47: "Последствия после Covid-19 и прививок на внешности",
                     48: "Волонтеры",
                     49: "Донорство для создания вакцины",
                     50: 'Вакцинация в Якутии',
                     51: 'Вакцинация в Нижегородской области',
                     52: 'Вакцинация в Архангельской области',
                     53: 'Вакцинация в Тверской области',
                     54: 'Вакцинация в Оренбурге и Новотроицке',
                     55: 'Вакцинация в Башкирии',
                     56: 'Вакцинация в Челябинске',
                     57: 'Вакцинация в Белгороде',
                     58: 'Вакцинация в Саратове',
                     59: 'Вакцинация в Дагестане',
                     60: 'Вакцинация в Карелии',
                     61: 'Вакцинация в Крыму',
                     62: "Региональные события"
                    }

In [ ]:
#создание столбца с укрупненными признаками
df['topic_concolidation'] = df['topic_finall_dv19 '].map(dict_dv19)

In [ ]:
#создание столбца с названиями топиков
df['topic_concolidation_category'] =df['topic_concolidation'].map(dict_categorydv19)

In [ ]:
#сохранение полученных данных
df.to_excel('finall_topic.xlsx')

#### Выделение метатопиков (укрупнение сгрупированных топиков в метатопики)

In [ ]:
metacategorydv19 = {'Разработка вакцины': 'Процесс создания вакцины от COVID-19',
                     'Мутация вируса SARS-CoV-2': 'Процесс создания вакцины от COVID-19',
                     'Донорство для создания вакцины': 'Процесс создания вакцины от COVID-19',                     
                     'Внешняя политика (международные отношения)': 'Вакцинация в мире', 
                     'Вакцинация в других странах': 'Вакцинация в мире', 
                     'Зарубежные вакцины': 'Вакцинация в мире', 
                     'Общая информация о вакцинации': 'Основные вопросы вакцинации',
                     'Доступность вакцин (доставка вакцин)': 'Основные вопросы вакцинации',
                     'Профилактика Covid-19': 'Основные вопросы вакцинации',
                     'Диагностика Covid-19': 'Основные вопросы вакцинации',
                     'Волонтеры': 'Основные вопросы вакцинации',
                     'Алкоголь при вакцинации': 'Основные вопросы вакцинации',
                     'Вакцинация от Covid-19 и гриппа': 'Основные вопросы вакцинации',
                     'Ревакцинация': 'Основные вопросы вакцинации',
                     'Антивакерский топик': 'Критика вакцинации',
                     'Последствия вакцинации': 'Критика вакцинации',
                     'Вакцинация и репродуктивное здоровье': 'Критика вакцинации',
                     'Последствия после Covid-19 и прививок на внешности': 'Критика вакцинации',
                     'Вакцинация в Оренбурге и Новотроицке': 'Вакцинация в регионах', 
                     'Вакцинация в Нижегородской области': 'Вакцинация в регионах',
                     'Региональные события': 'Вакцинация в регионах',
                     'Вакцинация в Саратове': 'Вакцинация в регионах',
                     'Вакцинация в Архангельской области': 'Вакцинация в регионах',
                     'Вакцинация в Тверской области': 'Вакцинация в регионах',
                     'Вакцинация в Якутии': 'Вакцинация в регионах',
                     'Вакцинация в Дагестане': 'Вакцинация в регионах',
                     'Вакцинация в Карелии': 'Вакцинация в регионах',
                     'Вакцинация в Челябинске': 'Вакцинация в регионах',
                     'Вакцинация в Белгороде': 'Вакцинация в регионах',
                     'Вакцинация в Крыму': 'Вакцинация в регионах',
                     'Вакцинация в Башкирии': 'Вакцинация в регионах',
                     'Вакцинация в школьных и дошкольных учреждениях': 'Вакцинация различных групп населения',
                     'Позиция верующих': 'Вакцинация различных групп населения',
                     'Вакцинация в высших учебных заведениях': 'Вакцинация различных групп населения',
                     'Вакцинация военных': 'Вакцинация различных групп населения',
                     'Вакцинации детей и подростков': 'Вакцинация различных групп населения',
                     'Официальная позиция Министерства здравоохранения': 'Позиция и действия властей',
                     'Политические партии': 'Позиция и действия властей',
                     'Официальная позиция Кремля': 'Позиция и действия властей',
                     'Призы и розыгрыши': 'Позиция и действия властей',
                     'Поощрение за прививку': 'Позиция и действия властей',
                     'Вакцинация в СМИ и социальных медиа' : 'Вакцинация в СМИ и социальных медиа',
                     'Общественный отношение (социологические опросы)': 'Вакцинация в СМИ и социальных медиа',
                     'Борьба с ложной информацией': 'Вакцинация в СМИ и социальных медиа',
                     'Общественные места': 'Эпидемиологические ограничения',
                     'Covid-паспорта и QR-коды': 'Эпидемиологические ограничения',
                     'Эффективность вакцины' : 'Эффективность вакцины',
                     'Мнения и высказывания экспертов' : 'Мнения и высказывания экспертов',
                     'Пункты вакцинации' : 'Пункты вакцинации',
                     'Экономика' : 'Экономика',
                     'Вакансии' : 'Вакансии',
                     'Принудительная вакцинация' : 'Принудительная вакцинация',
                     'Обсуждение вакцинации пользователями' : 'Обсуждение вакцинации пользователями',
                     'Туризм и путешествия' : 'Туризм и путешествия',
                     'Вакцинация животных от Covid-19' : 'Вакцинация животных от Covid-19',
                     'События, связанные с публичными личностями' : 'Вакцинация известных личностей',
                     'Правонарушения' : 'Правонарушения',
                     'Спорт и спортивные события' : 'Спорт и спортивные события',
                     'Эмиграция' : 'Эмиграция',
                     'Отстранение от работы' : 'Отстранение от работы',
                     'Провакерский топик':'Проваксерская позиция',
                     'Эпидемиологические ограничения' : 'Эпидемиологические ограничения'   
            }

In [ ]:
#создание столбца с метатопиками
df['topic_metacategory']  = df['topic_concolidation_category'] .map(metacategorydv19)

In [ ]:
#сохранение полученных данных
df.to_excel('finall_metatopic.xlsx')